# Logistic Regression

In this notebook you will use GPU-accelerated logistic regression to predict infection risk based on features of our population members.

## Objectives

By the time you complete this notebook you will be able to:

- Use GPU-accelerated logistic regression

## Imports

In [1]:
import cudf
import cuml

import cupy as cp

## Load Data

In [2]:
gdf = cudf.read_csv('./data/pop_2-05.csv', usecols=['age', 'sex', 'infected'])

In [3]:
gdf.dtypes

age         float64
sex         float64
infected    float64
dtype: object

In [10]:
gdf['age'].value_counts()

53.0    824404
51.0    821388
52.0    820676
54.0    816103
27.0    815488
         ...  
85.0    221588
86.0    204508
87.0    184122
88.0    161014
89.0    135301
Name: age, Length: 91, dtype: int32

In [16]:
gdf.head()

,age,sex,infected
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


## Logistic Regression

Logistic regression can be used to estimate the probability of an outcome as a function of some (assumed independent) inputs. In our case, we would like to estimate infection risk based on population members' age and sex.

Here we create a cuML logistic regression instance `logreg`:

In [17]:
logreg = cuml.LogisticRegression()

## Exercise: Regress Infected Status

The `logreg.fit` method takes 2 arguments: the model's independent variables *X*, and the dependent variable *y*. Fit the `logreg` model using the `gdf` columns `age` and `sex` as *X* and the `infected` column as *y*.

In [19]:
logreg.fit(gdf[['age','sex']],gdf['infected'])

LogisticRegression()

## Viewing the Regression

After fitting the model, we could use `logreg.predict` to estimate whether someone has more than a 50% chance to be infected, but since the virus has low prevalence in the population (around 1-2%, in this data set), individual probabilities of infection are well below 50% and the model should correctly predict that no one is individually likely to have the infection.

However, we also have access to the model coefficients at `logreg.coef_` as well as the intercept at `logreg.intercept_`. Both of these values are cuDF Series:

In [21]:
type(logreg.coef_)

cudf.core.series.Series

In [22]:
type(logreg.intercept_)

cudf.core.series.Series

Here we view these values. Notice that changing sex from 0 to 1 has the same effect via the coefficients as changing the age by ~48 years.

In [25]:
logreg_coef = logreg.coef_
logreg_int = logreg.intercept_

print("Coefficients: [age, sex]")
print([logreg_coef[0], logreg_coef[1]])

print("Intercept:")
print(logreg_int[0])

Coefficients: [age, sex]
[0.014860597365940085, 0.6956658839442457]
Intercept:
-5.222369426101315


## Estimate Probability of Infection

As with all logistic regressions, the coefficients allow us to calculate the logit for each; from that, we can calculate the estimated percentage risk of infection.

In [26]:
class_probs = logreg.predict_proba(gdf[['age', 'sex']])
class_probs

,0,1
0,0.994634,0.005366
1,0.994634,0.005366
2,0.994634,0.005366
3,0.994634,0.005366
4,0.994634,0.005366
...,...,...
58479889,0.960428,0.039572
58479890,0.960428,0.039572
58479891,0.960428,0.039572
58479892,0.960428,0.039572


Remembering that a 1 indicates 'infected', we assign that class' probability to a new column in the original dataframe:

In [27]:
gdf['risk'] = class_probs[1]

Looking at the original records with their new estimated risks, we can see how estimated risk varies across individuals.

In [28]:
gdf.take(cp.random.choice(gdf.shape[0], size=5, replace=False))

,age,sex,infected,risk
29511429,1.0,1.0,0.0,0.010859
37086474,24.0,1.0,0.0,0.015216
36595315,22.0,1.0,0.0,0.014777
13293938,35.0,0.0,0.0,0.008993
2034294,5.0,0.0,0.0,0.005777


## Exercise: Show Infection Prevalence is Related to Age

The positive coefficient on age suggests that the virus is more prevalent in older people, even when controlling for sex.

For this exercise, show that infection prevalence has some relationship to age by printing the mean `infected` values for the oldest and youngest members of the population when grouped by age:

In [41]:
grouped_by_age = gdf[['age','infected']].groupby('age')
print(grouped_by_age.mean().head())
print(grouped_by_age.mean().tail())

      infected
age           
66.0  0.020700
71.0  0.021292
82.0  0.022929
64.0  0.020675
77.0  0.022102
      infected
age           
33.0  0.015707
76.0  0.021928
74.0  0.021807
79.0  0.022518
86.0  0.023417


## Exercise: Show Infection Prevalence is Related to Sex

Similarly, the positive coefficient on sex suggests that the virus is more prevalent in people with sex = 1 (females), even when controlling for age.

For this exercise, show that infection prevalence has some relationship to sex by printing the mean `infected` values for the population when grouped by sex:

In [40]:
sex_groups = gdf[['sex', 'infected']].groupby(['sex'])
print(sex_groups.mean())

     infected
sex          
0.0  0.010140
1.0  0.020713


## Making Predictions with Separate Training and Test Data

cuML gives us a simple method for producing paired training/testing data:

In [43]:
X_train, X_test, y_train, y_test  = cuml.train_test_split(gdf[['age', 'sex']], gdf['infected'], train_size=0.9)

## Exercise: Fit Logistic Regression Model Using Training Data

For this exercise, create a new logistic regression model `logreg`, and fit it with the *X* and *y* training data just created.

In [46]:
logreg2 = cuml.LogisticRegression()
logreg2.fit(X_train,y_train)

LogisticRegression()

## Use Test Data to Validate Model

We can now use the same procedure as above to predict infection risk using the test data:

In [47]:
y_test_pred = logreg.predict_proba(X_test, convert_dtype=True)[1]
y_test_pred.index = X_test.index
y_test_pred

53966145    0.029275
52074079    0.026844
6580760     0.007000
52499798    0.027235
23081790    0.013179
              ...   
48120561    0.022889
13870672    0.009262
56079918    0.032380
53102351    0.028034
49378836    0.023907
Name: 1, Length: 5847990, dtype: float64

As we saw before, very few people are actually infected in the population, even among the highest-risk groups. As a simple way to check our model, we split the test set into above-average predicted risk and below-average predicted risk, then observe that the prevalence of infections correlates closely to those predicted risks.

In [48]:
test_results = cudf.DataFrame()
test_results['age'] = X_test['age']
test_results['sex'] = X_test['sex']
test_results['infected'] = y_test
test_results['predicted_risk'] = y_test_pred

test_results['high_risk'] = test_results['predicted_risk'] > test_results['predicted_risk'].mean()

risk_groups = test_results.groupby('high_risk')
risk_groups.mean()

,age,sex,infected,predicted_risk
high_risk,,,,
True,56.217058,0.889919,0.023769,0.023324
False,29.513484,0.252154,0.009909,0.010320


Finally, in a few milliseconds, we can do a two-tier analysis by sex and age:

In [49]:
%%time
s_groups = test_results[['sex', 'age', 'infected', 'predicted_risk']].groupby(['sex', 'age'])
s_groups.mean()

CPU times: user 13 ms, sys: 12.3 ms, total: 25.4 ms
Wall time: 24.1 ms


infected  predicted_risk
sex age                           
0.0 17.0  0.006870        0.006897
    13.0  0.005791        0.006502
1.0 18.0  0.015321        0.013936
0.0 39.0  0.010521        0.009539
1.0 42.0  0.023942        0.019791
...            ...             ...
    50.0  0.026017        0.022233
0.0 27.0  0.009689        0.007993
1.0 77.0  0.027044        0.032849
    54.0  0.025897        0.023563
0.0 29.0  0.009542        0.008232

[182 rows x 2 columns]

<br>
<div align="center"><h2>Please Restart the Kernel</h2></div>

In [50]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Next

In the next notebook, you will use GPU-accelerated k-nearest-neighbors algorithm to locate the nearest road nodes to each hospital.